In [1]:
__author__ = 'Alice Jacques <alice.jacques@noirlab.edu>, Astro Data Lab Team <datalab@noirlab.edu>' 
__version__ = '20230726' #yyyymmdd 
__datasets__ = ['ls_dr9', 'sdss_dr17', 'gaia_dr3', 'des_dr2'] 
__keywords__ = ['crossmatch', 'joint query', 'mydb', 'vospace']

# Examples using the pre-crossmatched tables at Astro Data Lab

by Alice Jacques and the Astro Data Lab Team

### Table of contents
* [Goals](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Authentication](#auth)
* [Writing a query with a JOIN statement](#joinquery)
* [Examples of using a JOIN statement in a query with LS and SDSS catalogs](#lssdss)
* [Saving results to VOSpace](#savetovospace)
* [Saving results to MyDB](#savetomydb)
* [Example Using a User Table and a pre-crossmatched Table](#usertable)
* [Resources & references](#refs)

<a class="anchor" id="goals"></a>
# Goals

* Learn how to write a query with a JOIN statement to retrieve information from a pre-crossmatched table and another Data Lab table
* Use a user-provided table to crossmatch a Data Lab table

For a more in-depth explanation of the pre-crossmatched tables hosted at Astro Data Lab, see our [How-to use pre-crossmatched tables notebook](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/CrossmatchTables/How_to_use_pre_crossmatched_tables.ipynb). 

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup

In [2]:
# std lib
from getpass import getpass

# 3rd party
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import pylab as plt
import matplotlib
%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert # converts table to Pandas dataframe object

<a class="anchor" id="auth"></a>
# Authentication
Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, un-comment the cell below and execute it:

In [3]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

'demo00'

<a class="anchor" id="joinquery"></a>
# Writing a query with a JOIN statement
In order to extract only the relevant columns pertaining to our science question from multiple data tables, we may write a query that uses a JOIN statement. There are 4 main types of JOIN statements that we could use, and which one we decide to choose depends on how we want the information to be extracted. 
1. **(INNER) JOIN**: Returns rows that have matching values in both tables
2. **LEFT (OUTER) JOIN**: Returns all rows from the left table, and the matched rows from the right table
3. **RIGHT (OUTER) JOIN**: Returns all rows from the right table, and the matched rows from the left table
4. **FULL (OUTER) JOIN**: Returns all rows when there is a match in either left or right table

Take a moment to look over the figure below outlining the various JOIN statement types.  
NOTE: the default JOIN is an `INNER JOIN`.

<img src='join.png'></img>

### `JOIN LATERAL`
In nearest neighbor crossmatch queries, we use `JOIN LATERAL`, which is like a SQL foreach loop that will iterate over each row in a result set and evaluate a subquery using that row as a parameter.

<a class="anchor" id="lssdss"></a>
# Examples of using a JOIN statement in a query with LS and SDSS catalogs
## Example 1: A Single JOIN
First we will examine the spectroscopic redshift of objects that are found in both the SDSS DR17 catalog and the LS DR9 catalog by writing a query with a single JOIN statement between their pre-crossmatched table and the SDSS DR17 table. The two crossmatch tables related to these two catalogs are:

`ls_dr9.x1p5__tractor__sdss_dr17__specobj`  
`sdss_dr17.x1p5__specobj__ls_dr9__tractor`

The choice of which of these two crossmatch tables to use should be based on the science question being posed. For instance, the question *'how does a galaxy's structure change with redshift?'* is dependent on the redshift values obtained from SDSS DR17, so we should use the crossmatch table that has SDSS DR17 as the first table. So, the relevant information we want to select from our two tables of interest for this example are:

1. "X" = `sdss_dr17.x1p5__specobj__ls_dr9__tractor`
    - **ra1** (RA of SDSS object)
    - **dec1** (Dec of SDSS object)
2. "S" = `sdss_dr17.specobj`
    - **z** (redshift)

### Write the single JOIN statement query
Now that we know what we want and where we want it from, let's write the query and then print the results on screen. Here we use one JOIN statement: it will search in the SDSS DR17 `specobj` table for rows that have the same SDSS id value (`specobjid`) as in the pre-crossmatched table (`id1`) and retrieve the desired columns from the SDSS DR17 `specobj` table within the specified RA and Dec region.

In [4]:
query_single = ("""
SELECT 
    X.ra1 AS ra_sdss, X.dec1 AS dec_sdss,
    S.z
FROM
    sdss_dr17.x1p5__specobj__ls_dr9__tractor AS X 
JOIN
    sdss_dr17.specobj AS S ON X.id1 = S.specobjid 
WHERE
    X.ra1 BETWEEN %s and %s and X.dec1 BETWEEN %s and %s
LIMIT 10000
"""
) %(110,200,7.,40.)  #large region
print(query_single) # print the query statement to screen


SELECT 
    X.ra1 AS ra_sdss, X.dec1 AS dec_sdss,
    S.z
FROM
    sdss_dr17.x1p5__specobj__ls_dr9__tractor AS X 
JOIN
    sdss_dr17.specobj AS S ON X.id1 = S.specobjid 
WHERE
    X.ra1 BETWEEN 110 and 200 and X.dec1 BETWEEN 7.0 and 40.0
LIMIT 10000



### Execute the single JOIN statement query and print results

In [5]:
%%time
df_single = qc.query(sql=query_single,fmt='pandas')
df_single

CPU times: user 36.8 ms, sys: 10.7 ms, total: 47.5 ms
Wall time: 256 ms


,ra_sdss,dec_sdss,z
0,110.00027,39.699210,0.516598
1,110.00065,39.613622,0.053511
2,110.00140,30.987794,-0.000062
3,110.00143,37.534445,-0.000253
4,110.70470,36.963974,0.000046
...,...,...,...
9995,113.19078,36.542547,0.328183
9996,113.19078,39.498064,0.693652
9997,113.28532,24.537772,1.648059
9998,113.19086,31.552450,0.170207


## Example 2: A Double JOIN
Now we will examine both the spectroscopic redshifts from SDSS DR17 and the photometry from LS DR9 by writing a query with two JOIN statements. The relevant information we want to select from our three tables of interest for this example are:

1. "X" = `sdss_dr17.x1p5__specobj__ls_dr9__tractor`
    - **ra1** (RA of SDSS object)
    - **dec1** (Dec of SDSS object)
2. "S" = `sdss_dr17.specobj`
    - **z** (redshift)
3. "L" = `ls_dr9.tractor`
    - **mag_g** (converted g magnitude)
    - **mag_r** (converted r magnitude)

### Write the double JOIN statement query
In this example we use two JOIN statements: the first will search in the SDSS DR17 `specobj` table for rows that have the same SDSS id value (`specobjid`) as in the pre-crossmatched table (`id1`) and retrieve the desired columns from the SDSS DR17 `specobj` table. The second will search in the LS DR9 `tractor` table for rows that have the same LS id value (`ls_id`) as in the pre-crossmatched table (`id2`) and retrieve the desired columns from the LS DR9 `tractor` table within the specified RA and Dec region.

In [6]:
query_double = ("""
SELECT 
    X.ra1 AS ra_sdss, X.dec1 AS dec_sdss,
    S.z,
    L.mag_g, L.mag_r
FROM
    sdss_dr17.x1p5__specobj__ls_dr9__tractor AS X 
JOIN
    sdss_dr17.specobj AS S ON X.id1 = S.specobjid 
JOIN
    ls_dr9.tractor AS L ON X.id2 = L.ls_id
WHERE
    X.ra1 BETWEEN %s and %s and X.dec1 BETWEEN %s and %s
LIMIT 10000
"""
) %(110,200,7.,40.)  #large region
print(query_double) # print the query statement to screen


SELECT 
    X.ra1 AS ra_sdss, X.dec1 AS dec_sdss,
    S.z,
    L.mag_g, L.mag_r
FROM
    sdss_dr17.x1p5__specobj__ls_dr9__tractor AS X 
JOIN
    sdss_dr17.specobj AS S ON X.id1 = S.specobjid 
JOIN
    ls_dr9.tractor AS L ON X.id2 = L.ls_id
WHERE
    X.ra1 BETWEEN 110 and 200 and X.dec1 BETWEEN 7.0 and 40.0
LIMIT 10000



### Execute the double JOIN statement query and print results

In [7]:
%%time
df_double = qc.query(sql=query_double,fmt='pandas')
df_double

CPU times: user 45.4 ms, sys: 11.1 ms, total: 56.6 ms
Wall time: 925 ms


,ra_sdss,dec_sdss,z,mag_g,mag_r
0,123.12650,39.993317,0.124905,18.130915,16.995148
1,123.36125,39.996015,-0.000117,21.921595,20.344702
2,124.93368,39.862601,0.174130,18.598770,17.707087
3,123.73187,39.973827,1.764000,21.709936,20.956276
4,123.83651,39.987596,0.463355,20.612232,18.626604
...,...,...,...,...,...
9995,141.17950,39.849704,1.112722,20.199490,19.831783
9996,138.24043,39.549354,1.303211,20.552681,20.535364
9997,140.28782,39.528155,0.183059,18.847853,18.380463
9998,141.04811,39.841038,0.000163,20.892084,20.232218


<a class="anchor" id="savetovospace"></a>
# Saving results to VOSpace
VOSpace is a convenient storage space for users to save their work. It can store any data or file type. We can save the results from a query to our virtual storage space. First, a basic query extracting a few thousand rows of the specobjid, ra, and dec columns from the SDSS DR17 specobj table:

In [8]:
basic_query = "SELECT specobjid, ra, dec FROM sdss_dr17.specobj LIMIT 10000"
print(basic_query)

SELECT specobjid, ra, dec FROM sdss_dr17.specobj LIMIT 10000


##### Submit the query, format the output as a CSV, and save it to VOSpace:

In [9]:
%%time
response = qc.query(sql=basic_query,fmt='csv',out='vos://basic_result.csv')

CPU times: user 24.9 ms, sys: 2.67 ms, total: 27.6 ms
Wall time: 1.51 s


##### Let's ensure the file was saved in VOSpace:

In [10]:
sc.ls(name='vos://basic_result.csv')

'basic_result.csv'

##### We will then remove the file from VOSpace:

In [11]:
sc.rm(name='vos://basic_result.csv')

'OK'

##### And ensure it was removed:

In [12]:
sc.ls(name='vos://basic_result.csv')

'A Node does not exist with the requested URI.'

<a class="anchor" id="savetomydb"></a>
# Saving results to MyDB
MyDB is a useful remote per-user relational database that can store data tables. Furthermore, the results of queries can be directly saved to MyDB, as we show in the following example:

In [13]:
%%time
response = qc.query(sql=basic_query, fmt='csv', out='mydb://basic_result', drop=True)

CPU times: user 19.3 ms, sys: 1.34 ms, total: 20.7 ms
Wall time: 320 ms


##### Ensure the table has been saved to MyDB by calling the `mydb_list()` function, which will list all tables currently in a user's MyDB:

In [14]:
print(qc.mydb_list(),"\n")

basic_result
bgsfaint_dlnotebook
desi_tile
df_xmatch
fastspec_everest_z_lt_0p6
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat,
testcm
testingx
testx2
twomasspsc
usno_objects
 



<a class="anchor" id="usertable"></a>
# Example: a User Table and a pre-crossmatched Table
Example: a user has only ID, RA, and Dec columns for their own table. They then perform a crossmatch between their table and `gaia_dr3.gaia_source`. In doing so, they can use the pre-crossmatched table `gaia_dr3.x1p5__gaia_source__ls_dr9__tractor` to get `ls_dr9.tractor` ID, RA, and Dec columns for free since the pre-crossmatched table gives them the corresponding `gaia_dr3.gaia_source` ID. We will use the `basic_result` table we stored to our MyDB in the previous section as our "user-provided" table.

##### Write the nearest-neighbor crossmatch query

In [15]:
query_xmatch = """
SELECT
    b.specobjid AS sdss_id, gg.source_id AS gaia_id, 
    (q3c_dist(b.ra,b.dec,gg.ra,gg.dec)*3600.0) AS dist_arcsec 
FROM
    mydb://basic_result AS b
LEFT JOIN LATERAL (
    SELECT g.* 
    FROM 
        gaia_dr3.gaia_source AS g
    WHERE
        q3c_join(b.ra, b.dec, g.ra, g.dec, 0.01)
    ORDER BY
        q3c_dist(b.ra,b.dec,g.ra,g.dec)
    ASC LIMIT 1
) AS gg ON true
"""
print(query_xmatch)


SELECT
    b.specobjid AS sdss_id, gg.source_id AS gaia_id, 
    (q3c_dist(b.ra,b.dec,gg.ra,gg.dec)*3600.0) AS dist_arcsec 
FROM
    mydb://basic_result AS b
LEFT JOIN LATERAL (
    SELECT g.* 
    FROM 
        gaia_dr3.gaia_source AS g
    WHERE
        q3c_join(b.ra, b.dec, g.ra, g.dec, 0.01)
    ORDER BY
        q3c_dist(b.ra,b.dec,g.ra,g.dec)
    ASC LIMIT 1
) AS gg ON true



##### Submit the crossmatch query and output to MyDB

In [16]:
%%time
df_xmatch = qc.query(sql=query_xmatch,out="mydb://df_xmatch",drop=True) # set drop=True to remove an already existing table from MyDB with the same name

CPU times: user 24 ms, sys: 1.27 ms, total: 25.3 ms
Wall time: 7.62 s


##### We can print the table by writing a query to MyDB

In [17]:
q = "select * from mydb://df_xmatch"
re = qc.query(sql=q,fmt='pandas')
re

,sdss_id,gaia_id,dist_arcsec
0,2889072702671316992,1.962451e+18,0.122596
1,2877815347283519488,1.962452e+18,0.135446
2,2877824143376541696,1.961010e+18,0.193541
3,2877823318742820864,1.960965e+18,0.133224
4,2889071878037596160,1.960965e+18,11.562321
...,...,...,...
9995,2857534859855816704,2.272961e+18,0.049671
9996,2866543439439226880,2.272962e+18,0.060718
9997,2857536234245351424,2.272973e+18,0.080895
9998,2866556633578760192,2.272977e+18,0.043344


##### We can now use the gaia_id column from our `df_xmatch` table above to get `ls_dr9.tractor` object information using the `gaia_dr3.x1p5__gaia_source__ls_dr9__tractor` pre-crossmatched table

In [18]:
%%time
query_1 = """
SELECT
    id2 AS ls_id, ra2 AS ls_ra, dec2 AS ls_dec,
    df_xmatch.gaia_id, df_xmatch.sdss_id
FROM
    gaia_dr3.x1p5__gaia_source__ls_dr9__tractor AS gxl
JOIN
    mydb://df_xmatch AS df_xmatch ON gxl.id1 = df_xmatch.gaia_id
"""
print(query_1)


SELECT
    id2 AS ls_id, ra2 AS ls_ra, dec2 AS ls_dec,
    df_xmatch.gaia_id, df_xmatch.sdss_id
FROM
    gaia_dr3.x1p5__gaia_source__ls_dr9__tractor AS gxl
JOIN
    mydb://df_xmatch AS df_xmatch ON gxl.id1 = df_xmatch.gaia_id

CPU times: user 101 µs, sys: 84 µs, total: 185 µs
Wall time: 142 µs


##### Submit the query and print the resulting table

In [19]:
%%time
df = qc.query(sql=query_1,fmt='pandas')
df

CPU times: user 33.5 ms, sys: 2.69 ms, total: 36.1 ms
Wall time: 326 ms


,ls_id,ls_ra,ls_dec,gaia_id,sdss_id
0,9907737095837650,287.228165,48.064735,2131102058617608064,3384465917919389696
1,9907737159009377,287.448870,48.229697,2131098455144685056,3384466192797296640
2,9907737158950166,287.387517,48.168933,2131097218194075904,3384462344506599424
3,9907737221862367,287.697861,48.382752,2131196930154414592,3384463718896134144
4,9907737221859120,287.547174,48.407548,2131195006009050240,3384465093285668864
...,...,...,...,...,...
840,9907740343599562,276.670899,63.135065,2160347319866203008,2873374430422132736
841,9907740343599573,276.671625,63.208500,2160347899685623680,2873378278712829952
842,9907740343599244,276.634148,63.135310,2160347972701229440,2873379103346550784
843,9907740343535831,276.356517,63.279633,2160352916207452416,2873378828468643840


<a class="anchor" id="refs"></a>
# Resources & references

W3Schools: SQL Joins https://www.w3schools.com/sql/sql_join.asp  